In [1]:
import os

In [2]:
%pwd

'/Users/himanshuagarwal/Desktop/Text-Summarizer/research'

In [3]:
%cd /Users/himanshuagarwal/Desktop/Text-Summarizer

/Users/himanshuagarwal/Desktop/Text-Summarizer


/opt/anaconda3/envs/textS/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%pwd

'/Users/himanshuagarwal/Desktop/Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformtion_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transforamtion_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transforamtion_config


In [8]:
import os
from textSummarizer.logger import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/opt/anaconda3/envs/textS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-08-16 22:24:11,639: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformtion_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-08-16 22:24:46,867: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-16 22:24:46,869: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-16 22:24:46,870: INFO: common: created directory at: artifacts]


AttributeError: 'ConfigurationManager' object has no attribute 'get_data_transformation_config'